In [4]:
import pandas as pd
from pandas import json_normalize

import requests 
import json
import os
from dotenv import load_dotenv 
from bs4 import BeautifulSoup
import re
import time


# Web scraping for minimum wage in Turkey throughout the years


In [28]:
url = 'https://countryeconomy.com/national-minimum-wage/turkey'
res = requests.get(url)
html = res.content
soup = BeautifulSoup(html, "html.parser")

In [66]:
y = soup.select("td.fecha")
y[0].getText().strip()

year = [i.getText() for i in y]
# year = [i for i in year if 'June' not in i]
year[:3]

['June 2022', 'January 2022', 'January 2021']

In [67]:
mw = soup.find_all("td", attrs={"class":"numero"})
mw[0].getText().strip()

min_wage = [i.getText() for i in mw]
min_wage = [i for i in min_wage if '$' not in i and '€' not in i]

min_wage[:3]

['6,471.0', '5,004.0', '3,577.5']

In [68]:
mwu = soup.select("td.numero.dol")
mwu[0].getText().strip()

min_wage_usd = [i.getText() for i in mwu]
min_wage_usd[:3]

['$394.5', '$375.1', '$404.2']

In [69]:
turkey = {
    'year': year,
    'min_wage': min_wage,
    'min_wage_dollar': min_wage_usd
}

turkey_wages = pd.DataFrame(turkey)
turkey_wages = turkey_wages[turkey_wages['year'].str.contains('June')==False]


turkey_wages = turkey_wages.replace('January ', "", regex = True).replace(',', '', regex = True).replace('\$', '', regex=True)
turkey_wages = turkey_wages.astype({'min_wage' : float, 'min_wage_dollar' : float, 'year': int})

turkey_wages.sample(3)


,year,min_wage,min_wage_dollar
30,2003,306.0,203.9
1,2022,5004.0,375.1
13,2013,978.6,536.6


# Cleaning the price variation dataset and merging the two

In [50]:
# Reading the dataset used for this anylisis.
# "global_prices" is a dataset displaying the price variation for different products in 76 countries from 2002 to 2021.

global_prices = pd.read_csv(".\Data\global_food_prices.csv", encoding='unicode_escape')
global_prices.sample(5)

C:\Users\mayar\AppData\Local\Temp\ipykernel_19320\1891304889.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  global_prices = pd.read_csv(".\Data\global_food_prices.csv", encoding='unicode_escape')


,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
1137794,162.0,Mexico,2048,Puebla,2626,Puebla,67,Maize (white) - Wholesale,0.0,MXN,14,Wholesale,5,KG,6,2006,2.40,NaN
922661,141.0,Lebanon,1798,NaN,1689,Beirut,341,Fuel (gas) - Retail,0.0,LBP,15,Retail,83,10 KG,12,2019,14400.00,NaN
1777706,238.0,Syrian Arab Republic,2840,Hama,1136,Hama,334,Salt (iodised) - Retail,0.0,SYP,15,Retail,5,KG,6,2021,600.00,NaN
1224355,175.0,Nepal,2153,Eastern,646,Biratnagar,81,Milk - Retail,0.0,NPR,15,Retail,15,L,1,2020,80.00,NaN
133787,115.0,Bassas da India,1504,NaN,4668,Jeypore,325,Lentils (moong) - Retail,0.0,INR,15,Retail,5,KG,11,2020,97.13,NaN


In [51]:
# Creating a subset of the price variation that only takes into consideration Turkey.

prices_turkey = global_prices.loc[global_prices['adm0_name'] == "Turkey"]


In [52]:
# Since this analysis is focusing on the consumer, the prices for wholesale are not relevant.

prices_turkey = prices_turkey[prices_turkey['pt_name'].str.contains('Wholesale')==False]

prices_turkey = prices_turkey.replace(' - Retail', "", regex=True)


In [53]:

# Cleaning the table by dropping columns that are not relevant for the analysis and renaming the other ones 
# for readability and clarity.

prices_turkey.drop(['cur_id', 'cur_name', 'pt_id', 'um_id', 'mp_commoditysource', 'adm1_name'], axis = 'columns', inplace=True)

prices_turkey.drop(['cm_id', 'adm0_id', 'adm1_id', 'mkt_id', 'mkt_name', 'pt_name'], axis = 'columns', inplace=True)

prices_turkey.columns = prices_turkey.columns.str.replace('mp_', "")

prices_turkey.sample(5)



,adm0_name,cm_name,um_name,month,year,price
1875715,Turkey,Bananas,KG,10,2018,9.6345
1870828,Turkey,Coffee (instant),Unit,4,2020,35.0813
1869792,Turkey,Cocoa (powder),KG,11,2020,64.2024
1871755,Turkey,Eggplants,KG,6,2019,3.2300
1874957,Turkey,Lentils,KG,12,2017,7.5663


In [55]:
# Merging both dataframes by the year and creating a new one called products_wage_turkey that has information
# about the price variation of certain goods as well as the variation of the minimum wage in Turkey.

products_wage_turkey = prices_turkey.merge(turkey_wages, how='inner', on='year')
products_wage_turkey.sample(3)


,adm0_name,cm_name,um_name,month,year,price,min_wage,min_wage_dollar
10366,Turkey,Bulgur,KG,4,2021,6.1397,3577.5,404.2
5577,Turkey,Transport (public),Course,9,2019,2.4052,2558.4,450.9
551,Turkey,Sugar,KG,6,2015,3.7300,1201.5,468.2


In [56]:
# More cleaning for readability (renaming columns and rounding values).

products_wage_turkey.rename(
    columns =
    {'cm_name':'product', 'adm0_name':'country','um_name':'unit'},
    inplace = True
)

products_wage_turkey['price'] = products_wage_turkey['price'].round(2)


products_wage_turkey.sample(2)


,country,product,unit,month,year,price,min_wage,min_wage_dollar
6780,Turkey,Zucchini,KG,3,2019,7.25,2558.4,450.9
863,Turkey,Fish (fresh),KG,11,2015,14.26,1201.5,468.2


In [57]:
# Filtering the products by price and just taking the highest price of each year (instead of a price per month)

products_wage_filtered = products_wage_turkey.loc[products_wage_turkey.groupby(['year', 'product'])['price'].idxmax()]
products_wage_filtered.sort_values(by = ['product', 'year']).sample(4)


,country,product,unit,month,year,price,min_wage,min_wage_dollar
432,Turkey,Oranges,KG,10,2014,2.33,1071.0,518.2
2922,Turkey,Transport (public),Course,1,2017,2.95,1777.5,487.2
727,Turkey,Apples (red),KG,6,2015,2.98,1201.5,468.2
818,Turkey,Tomatoes (paste),KG,6,2015,6.81,1201.5,468.2


In [58]:
products_wage_filtered['product'].unique()


array(['Apples (red)', 'Bananas', 'Beans (white)', 'Bread (common)',
       'Bulgur', 'Cabbage', 'Cauliflower', 'Chickpeas', 'Cocoa (powder)',
       'Coffee', 'Coffee (instant)', 'Cucumbers (greenhouse)', 'Eggs',
       'Fish (fresh)', 'Garlic', 'Groundnuts (shelled)', 'Lentils',
       'Meat (chicken)', 'Meat (mutton)', 'Meat (veal)',
       'Milk (pasteurized)', 'Oil (olive)', 'Oil (sunflower)', 'Onions',
       'Oranges', 'Pasta', 'Peas (green, dry)', 'Rice', 'Salt', 'Sugar',
       'Tea', 'Tea (green)', 'Tomatoes', 'Wheat flour', 'Bread (pita)',
       'Cheese', 'Eggplants', 'Fuel (gas)', 'Fuel (petrol-gasoline)',
       'Milk (powder, infant formula)', 'Potatoes', 'Spinach',
       'Tomatoes (paste)',
       'Wage (non-qualified labour, non-agricultural)', 'Yogurt',
       'Apples', 'Cucumbers', 'Electricity', 'Tea (herbal)',
       'Transport (public)', 'Water', 'Zucchini'], dtype=object)

In [79]:
# The food products are the focus of this analysis, so everything that is not food should be filtered out.

food_wage_filtered = products_wage_filtered[(products_wage_filtered['product'] != 'Wage (non-qualified labour, non-agricultural)') & 
                     (products_wage_filtered['product'] != 'Electricity') & (products_wage_filtered['product'] !='Fuel (gas)') &
                     (products_wage_filtered['product'] != 'Fuel (petrol-gasoline)') & (products_wage_filtered['product'] != 'Transport (public)')]
                                                                                        

food_wage_filtered['product'].unique()

array(['Apples (red)', 'Bananas', 'Beans (white)', 'Bread (common)',
       'Bulgur', 'Cabbage', 'Cauliflower', 'Chickpeas', 'Cocoa (powder)',
       'Coffee', 'Coffee (instant)', 'Cucumbers (greenhouse)', 'Eggs',
       'Fish (fresh)', 'Garlic', 'Groundnuts (shelled)', 'Lentils',
       'Meat (chicken)', 'Meat (mutton)', 'Meat (veal)',
       'Milk (pasteurized)', 'Oil (olive)', 'Oil (sunflower)', 'Onions',
       'Oranges', 'Pasta', 'Peas (green, dry)', 'Rice', 'Salt', 'Sugar',
       'Tea', 'Tea (green)', 'Tomatoes', 'Wheat flour', 'Bread (pita)',
       'Cheese', 'Eggplants', 'Milk (powder, infant formula)', 'Potatoes',
       'Spinach', 'Tomatoes (paste)', 'Yogurt', 'Apples', 'Cucumbers',
       'Tea (herbal)', 'Water', 'Zucchini'], dtype=object)

In [224]:
# Creating a subset of the variation in minimum wage in Turkish lira (national currency) and dollars.
 
wage_variation = pd.DataFrame()

wage_variation.insert(0,'year', food_wage_filtered['year'].unique(), True)
wage_variation.insert(1,'min_wage', food_wage_filtered['min_wage'].unique(), True)
wage_variation.insert(2,'min_wage_dollar', food_wage_filtered['min_wage_dollar'].unique(), True)
wage_variation.insert(3,'%_var_wage', round((wage_variation['min_wage'].pct_change())*100, 1), True)
wage_variation.insert(4,'%_var_wage_usd', round((wage_variation['min_wage_dollar'].pct_change())*100, 1), True)

wage_variation

,year,min_wage,min_wage_dollar,%_var_wage,%_var_wage_usd
0,2013,978.6,536.6,NaN,NaN
1,2014,1071.0,518.2,9.4,-3.4
2,2015,1201.5,468.2,12.2,-9.6
3,2016,1647.0,545.3,37.1,16.5
4,2017,1777.5,487.2,7.9,-10.7
5,2018,2029.5,420.3,14.2,-13.7
6,2019,2558.4,450.9,26.1,7.3
7,2020,2943.0,419.9,15.0,-6.9
8,2021,3577.5,404.2,21.6,-3.7


In [226]:

food_price_variation_2015 = food_wage_filtered[(food_wage_filtered['year'] == 2015)][["product", "price"]]
food_price_variation_2015.reset_index(drop=True).head(4)


,product,price
0,Apples (red),2.98
1,Bananas,5.88
2,Beans (white),7.66
3,Bread (common),3.14


In [227]:

food_price_variation_2021 = food_wage_filtered[food_wage_filtered['year'] == 2021][["product", "price"]]
food_price_variation_2021.reset_index(drop=True).head(4)

,product,price
0,Apples,6.29
1,Bananas,16.19
2,Beans (white),17.72
3,Bread (common),8.40


In [239]:
food_price_variation  = food_price_variation_2015.merge(food_price_variation_2021, on="product")
food_price_variation["%_var_price"] = food_price_variation.apply(lambda x: round((x['price_y'] - x['price_x'])/x['price_x']*100, 1), axis=1)

food_price_variation.head(4)

,product,price_x,price_y,%_var_price
0,Bananas,5.88,16.19,175.3
1,Beans (white),7.66,17.72,131.3
2,Bread (common),3.14,8.40,167.5
3,Bulgur,2.73,6.35,132.6


In [235]:
# food_price_variation.insert(3,'%_var_price', round((food_price_variation.pct_change(axis = 'columns'))*100, 1), True)
# food_price_variation

TypeError: unsupported operand type(s) for /: 'float' and 'str'

In [238]:
# def variation(df, namecol, col1, col2):
    
#     df[namecol] = df.apply(lambda x: round((x['col1'] - x['col2'])/x['col2']*100, 1), axis=1)
    
#     return df

# food_wage_variation = variation(food_wage_variation, "%_var_price", 'price_y', 'price_x')



KeyError: 'price_y'

In [437]:
food_price_variation_top10 = food_wage_variation.sort_values(by="%_var_price", ascending=False)[:10]
food_price_variation_top10.reset_index(drop=True)

,product,%_var_price
0,Fish (fresh),287.8
1,Eggplants,214.0
2,Oranges,194.0
3,Tomatoes,188.4
4,Meat (mutton),182.0
5,Bananas,175.3
6,Eggs,175.0
7,Garlic,167.6
8,Bread (common),167.5
9,Meat (chicken),159.0


In [438]:
# Renaming the products to fit the API standard
food_price_variation_top10['product'] = food_price_variation_top10['product'].astype(str) + ', raw'

food_price_variation_top10 = food_price_variation_top10.replace('Bread (common), raw', 'Bread').replace('Meat ', '', regex=True)
food_price_variation_top10 = food_price_variation_top10.reset_index(drop=True)
food_price_variation_top10

,product,%_var_price
0,"Fish (fresh), raw",287.8
1,"Eggplants, raw",214.0
2,"Oranges, raw",194.0
3,"Tomatoes, raw",188.4
4,"(mutton), raw",182.0
5,"Bananas, raw",175.3
6,"Eggs, raw",175.0
7,"Garlic, raw",167.6
8,Bread,167.5
9,"(chicken), raw",159.0


In [459]:
food_price_variation_bottom10 = food_wage_variation.sort_values(by="%_var_price", ascending=True)[:10]
food_price_variation_bottom10 = food_price_variation_bottom10.reset_index(drop=True)
food_price_variation_bottom10

,product,%_var_price
0,Oil (sunflower),-10.0
1,Onions,38.5
2,Sugar,52.4
3,Oil (olive),55.4
4,Tomatoes (paste),59.6
5,Potatoes,60.2
6,Coffee,78.5
7,Cauliflower,79.3
8,Spinach,79.5
9,Chickpeas,82.7


# Using an API that checks the nutritional information of different foods

In [460]:
def api_codes(df):

    food_codes = []
    url = "https://food-nutrition-information.p.rapidapi.com/foods/search"

    for i in df['product']:

        querystring = {"query": i, "pageSize":"1","pageNumber":"1"}

        headers = {
            "X-RapidAPI-Key": "f5bc8265eemshffa75f77ba1b130p15dd15jsnf1b9068994ab",
            "X-RapidAPI-Host": "food-nutrition-information.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)
        food = response.json()

        codes = json_normalize(food)

        food_codes.append(codes['foods'][0][0]['fdcId'])
    
    return food_codes


In [461]:
def api_food_info(food_codes):

    food_info = dict()
    list_of_dfs = []
    foods_info_final = pd.DataFrame()


    for i in food_codes:

        food_info = dict()
        url = f"https://food-nutrition-information.p.rapidapi.com/food/{i}"

        headers = {
            "X-RapidAPI-Key": "f5bc8265eemshffa75f77ba1b130p15dd15jsnf1b9068994ab",
            "X-RapidAPI-Host": "food-nutrition-information.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers)
        info = response.json()

        # Variable 'a' will save the name of the food being registered using the key 'description' in the API dictionary
        a = str(json_normalize(info)['description'])
        a = a.replace('\nName: description, dtype: object', '')

        # 'food_info' will save all the nutritional info for each food in 'food_codes'
        food_info = json_normalize(info)['foodNutrients'][0]
        food_info = pd.DataFrame(food_info)

        # Creating a new columns called 'food' to register the variable 'a'
        # The same name will be repeated throughout this df so when all the food_info df are concatenated we'll have a way
        # of knowing which food has each piece of info
        food_info = food_info.assign(food = a)

        # Getting only the name of the nutrient and its unit
        food_info['nutrients'] = food_info['nutrient'].apply(lambda x : x['name']) 
        food_info['unit'] = food_info['nutrient'].apply(lambda x : x['unitName'])

        # Concatenating the dataframes together
        list_of_dfs.append(food_info)
        foods_info_final = pd.concat(list_of_dfs)



    return foods_info_final

In [462]:
def food_info_df(df):

    df.drop(['nutrient', 'type', 'id', 'dataPoints', 'foodNutrientDerivation'], axis = 'columns', inplace=True)
    df.drop(['max', 'min', 'median', 'minYearAcquired', 'nutrientAnalysisDetails', 'loq'], axis = 'columns', inplace=True)

    food_info2 = df[df['nutrients'].isin(['Energy','Carbohydrates', 'Sugars, total including NLEA', 'Total lipid (fat)'])]

    food_info2.insert(3, 'value', food_info2.pop('amount'))
    food_info2.reset_index(drop=True)

    return food_info2

In [463]:
food_codes_bottom10 = api_codes(food_price_variation_bottom10)
foods_info_bottom10 = api_food_info(food_codes_bottom10)


In [464]:
food_codes_top10 = api_codes(food_price_variation_top10)
foods_info_top10 = api_food_info(food_codes_top10)

In [465]:
info_top10 = food_info_df(foods_info_top10)
info_bottom10 = food_info_df(foods_info_bottom10)

info_top10

,food,nutrients,unit,value
2,"0 Fish, tuna, fresh, bluefin, raw",Energy,kcal,144.00
3,"0 Fish, tuna, fresh, bluefin, raw",Energy,kJ,602.00
5,"0 Fish, tuna, fresh, bluefin, raw",Total lipid (fat),g,4.90
7,"0 Fish, tuna, fresh, bluefin, raw",Carbohydrates,g,NaN
10,"0 Fish, tuna, fresh, bluefin, raw","Sugars, total including NLEA",g,0.00
2,"0 Eggplant, raw",Energy,kcal,25.00
4,"0 Eggplant, raw",Total lipid (fat),g,0.18
5,"0 Eggplant, raw",Carbohydrates,g,NaN
8,"0 Eggplant, raw","Sugars, total including NLEA",g,3.53
2,"0 Orange, raw",Energy,kcal,47.00


In [466]:
info_bottom10

,food,nutrients,unit,value
2,"0 Onions, raw",Energy,kcal,40.00
4,"0 Onions, raw",Total lipid (fat),g,0.10
5,"0 Onions, raw",Carbohydrates,g,NaN
8,"0 Onions, raw","Sugars, total including NLEA",g,4.24
1,0 SUGAR,Total lipid (fat),g,0.00
2,0 SUGAR,"Sugars, total including NLEA",g,100.00
5,0 SUGAR,Energy,kcal,375.00
2,0 Olive oil,Energy,kcal,884.00
4,0 Olive oil,Total lipid (fat),g,100.00
5,0 Olive oil,Carbohydrates,g,NaN
